In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import ImageDataGenerator

In [3]:
dataset_path = 'C://Users//ISHIKA JAIN//Downloads//archive (1)//Indian Food Images//Indian Food Images'

In [4]:
import os
from PIL import Image

In [5]:
# Define the image dimensions
img_width, img_height = 224, 224

# Load the images and labels
images = []
labels = []
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        label = folder
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if file.endswith('.jpg') or file.endswith('.png'):
                img = Image.open(file_path)
                img = img.resize((img_width, img_height))
                images.append(np.array(img))
                labels.append(label)

In [6]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split data into training and testing sets
n_samples = len(images)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [7]:
# Data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode = 'nearest')

In [8]:
# Train the model
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

In [9]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
food_output = Dense(1, activation='softmax', name='food_output')(x)
calorie_output = Dense(1, activation='linear', name='calorie_output')(x)

model = Model(inputs=base_model.input, outputs=[food_output, calorie_output])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'food_output': 'categorical_crossentropy', 'calorie_output': 'mean_squared_error'},
              metrics={'food_output': 'accuracy', 'calorie_output': 'mean_absolute_error'})
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder to your labels and transform them to numerical labels
train_labels_encoded = le.fit_transform(train_labels)
test_labels_encoded = le.transform(test_labels)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels_encoded))
train_dataset = train_dataset.batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels_encoded))
test_dataset = test_dataset.batch(32)

history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10


C:\Users\ISHIKA JAIN\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\ISHIKA JAIN\anaconda3\Lib\site-packages\keras\src\losses\losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


  1/100 ━━━━━━━━━━━━━━━━━━━━ 1:10:14 43s/step - food_output_accuracy: 0.0000e+00 - loss: 0.0000e+00

C:\Users\ISHIKA JAIN\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 2027s 20s/step - food_output_accuracy: 0.0087 - loss: 0.0000e+00 - val_food_output_accuracy: 0.0188 - val_loss: 0.0000e+00
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2385s 24s/step - food_output_accuracy: 0.0087 - loss: 0.0000e+00 - val_food_output_accuracy: 0.0188 - val_loss: 0.0000e+00
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1820s 18s/step - food_output_accuracy: 0.0087 - loss: 0.0000e+00 - val_food_output_accuracy: 0.0188 - val_loss: 0.0000e+00
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1756s 18s/step - food_output_accuracy: 0.0087 - loss: 0.0000e+00 - val_food_output_accuracy: 0.0188 - val_loss: 0.0000e+00
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1926s 19s/step - food_output_accuracy: 0.0087 - loss: 0.0000e+00 - val_food_output_accuracy: 0.0188 - val_loss: 0.0000e+00
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1562s 16s/step - food_output_accuracy: 0.0087 - loss: 0.0000e+00 - val_food_output_accuracy: 0.0188 - val_loss: 0.0000e+00
Epoch 7/10
100/100 ━━━━━━━━━━━━━━

In [15]:
# Evaluate the model , 
food_accuracy, calorie_mae = model.evaluate(test_dataset)
#print(f'Test loss: {loss:.3f}')
print(f'Test food accuracy: {food_accuracy:.3f}')
print(f'Test calorie MAE: {calorie_mae:.3f}')

# Make predictions
predictions = model.predict(test_dataset)

# Unpack the predictions into food and calorie predictions
food_predictions, calorie_predictions = predictions

# Print some predictions
# Iterate over the predictions and print them
for i in range(5):
    # Get the predicted food class
    predicted_food_class = np.argmax(food_predictions[i])
    
    # Get the predicted calorie value
    predicted_calorie_value = calorie_predictions[i][0]
    
    # Print the predictions
    print(f'Food prediction: {le.inverse_transform([predicted_food_class])}')
    print(f'Calorie prediction: {predicted_calorie_value:.2f}')
    print('---')


25/25 ━━━━━━━━━━━━━━━━━━━━ 117s 5s/step - food_output_accuracy: 0.0163 - loss: 0.0000e+00
Test food accuracy: 0.000
Test calorie MAE: 0.019
25/25 ━━━━━━━━━━━━━━━━━━━━ 117s 5s/step
Food prediction: ['adhirasam']
Calorie prediction: 55004.27
---
Food prediction: ['adhirasam']
Calorie prediction: 52830.86
---
Food prediction: ['adhirasam']
Calorie prediction: 43506.91
---
Food prediction: ['adhirasam']
Calorie prediction: 32937.86
---
Food prediction: ['adhirasam']
Calorie prediction: 28855.29
---
